# Simple script to input, calculate, and output

Note: in Jupyter notebooks, we can make comments in Markdown cells.  So making inline comments (lines that start with the hash character # and are ignored by Python) is less important.  

Here we set up the data needed to run the calculation

In [ ]:
# This is an inline comment that will be ignored by Python
pi = 3.14159
print(pi)
diameter = float(input('Enter the diameter, then press the Enter/return key: '))  # when this line runs you need to enter a number
print(diameter)

The data need to be transformed into a usable form

In [ ]:
radius = diameter/2
print(radius)

We have all of the information necessary to complete the calculation

In [ ]:
print('The area of the circle is ',pi*radius**2)

# Using an API to plot the current location of the International Space Station

In [ ]:
import requests
import webbrowser

url = 'http://api.open-notify.org/iss-now.json'
response = requests.get(url)
print('JSON text: ', response.text)
print()
data = response.json()
print('Python data structure: ', data)
print()
latitude = data['iss_position']['latitude']
longitude = data['iss_position']['longitude']
zoom = '4'
googleMapUrl = 'http://www.google.com/maps/place/'+latitude+','+longitude+'/@'+latitude+','+longitude+','+zoom+'z'
print(googleMapUrl)
# The following line will open a tab on your browser with the map if you run it in a local Jupiter notebook (but not in Colab)
success = webbrowser.open_new_tab(googleMapUrl)


# Named Entity Recognition (NER)

There are a number of ways to do NER. This script uses the powerful Natural Language Toolkit (nltk).

The first step is to load several libraries and download data needed by the apply the model.


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import json

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

print('done')

The next step is to run the example text through a pipeline that breaks the text into words, flags the words by part of speech, then tags chunks of words that are thought to be named entites.

In [ ]:
text = '''The House committee investigating the Jan. 6 attack on the U.S. Capitol on Tuesday evening unanimously approved a criminal contempt report against Steve Bannon, an ally of former President Donald Trump's, for defying a subpoena from the panel.'''

tokens = nltk.word_tokenize(text)
print('tokens:', tokens)
tagged_tokens = nltk.pos_tag(tokens)
print('tagged:', tagged_tokens)
named_entity_chunks = nltk.ne_chunk(tagged_tokens)
print('Named Entity chunks:', named_entity_chunks)
print()


In this last step, we extract data from the pipeline output for human-friendly display and collect it in a Python data structure for further use.

In [ ]:
ne_list = []
for chunk in named_entity_chunks:
    if hasattr(chunk, 'label'):
        ne_dict = {'ne_label': chunk.label()}
        # A chunk is some kind of iterable of tuples
        # Each tuple contains (word, noun_descriptor)
        ne_string = chunk[0][0] # 0th tuple, word
        # Iterate through the rest of the tuples in the chunk
        for additional_tuple in chunk[1:len(chunk)]:
            ne_string += ' ' + additional_tuple[0]
        ne_dict['ne_string'] = ne_string
        ne_list.append(ne_dict)

        print(chunk.label(), ' '.join(c[0] for c in chunk))
print()
print('NE list:', json.dumps(ne_list, indent = 2))

# Face detection using pretrained machine learning model using OpenCV

The cv2 module is an open source computer vision library.

We will use this image for a test:

![Vanderbilt police car](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Beatles_ad_1965_just_the_beatles_crop.jpg/320px-Beatles_ad_1965_just_the_beatles_crop.jpg)

EMI., Public domain, via Wikimedia Commons

**NOTE: After running this code, it's best not to save the notebook without first clearing the image output (`Edit` menu, then select `Clear all outputs`). Without clearing the output, the image data gets saved with the notebook and the time to save may be long.**

In the first section of code, we load a number of modules and define two functions that we will use later on.

In [ ]:
# Import libraries and define functions

import numpy as np
import urllib.request as urllib
import cv2 
import json
import matplotlib.pyplot as plt
import requests

%matplotlib inline

def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def detect_faces(cascade, test_image, scaleFactor = 1.1):
    # create a copy of the image to prevent any changes to the original one.
    image_copy = test_image.copy()
    
    #convert the test image to gray scale as opencv face detector expects gray images
    gray_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
       
    # Applying the haar classifier to detect faces
    faces_rect = cascade.detectMultiScale(gray_image, scaleFactor=scaleFactor, minNeighbors=5)
    
    for (x, y, w, h) in faces_rect:
        cv2.rectangle(image_copy, (x, y), (x+w, y+h), (0, 255, 0), 15)

    image_dimensions = test_image.shape # .shape gives height, width, and layers

    # Create a Python data structure for the face rectangles discovered
    faces_dict = {'image_width': str(image_dimensions[1]), 'image_height': str(image_dimensions[0])}
    faces_list = []
    for i in faces_rect:
        faces_list.append({'x': str(i[0]), 'y': str(i[1]), 'width': str(i[2]), 'height': str(i[3])})
    faces_dict['faces'] = faces_list
        
    return image_copy, faces_dict

The training data is available on the web, but cv2 expects that the training data are in a local file. So we grab the data from GitHub, save the file in the current working directory on the cloud server, then load the file to create a classifier object.

There are 4 pretrained models that we can try by uncommenting them in the first section of code (delete the `#` sign from the one you want to use and insert `#` in front of the one you are finished with).



In [ ]:
# Uncomment to try a different training model
training_file = 'haarcascade_frontalface_default.xml'
#training_file = 'haarcascade_frontalface_alt.xml'
#training_file = 'haarcascade_frontalface_alt2.xml'
#training_file = 'haarcascade_frontalface_alt_tree.xml'

# Load traning model from GitHub
training_data_url = 'https://github.com/parulnith/Face-Detection-in-Python-using-OpenCV/raw/master/data/haarcascades/' + training_file
training_data = requests.get(training_data_url).text

# Save file on cloud server
with open(training_file, 'wt', encoding='utf-8') as file_object:
    file_object.write(training_data)

haar_cascade_face = cv2.CascadeClassifier(training_file)


The final section of code retrieves the image from Wikimedia Commons via a URL, then loads it as a cv2 image. (The commented out code can be used if you are running the notebook locally and want to load the file from a path in your filesystem.)

The `detect_faces()` function runs the machine learning model on the picture. The last line displays the image with rectangles superimposed to show any faces that were detected. The raw pixel rectangle data are available in the `faces_dict`, which you can view by uncommenting the print statement.

In [ ]:
# URL of test image
image_url = 'https://upload.wikimedia.org/wikipedia/commons/9/9f/Beatles_ad_1965_just_the_beatles_crop.jpg'

#get image by url and load into cv2
resp = urllib.urlopen(image_url)
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

# alternate code to load image from a local file
# image = cv2.imread(path)

#call the function to detect faces
faces, faces_dict = detect_faces(haar_cascade_face, image)

# uncomment to see the faces rectangles metadata
#print(json.dumps(faces_dict, indent = 2))

#convert to RGB and display image
plt.imshow(convertToRGB(faces))